### Code Hist.

 - CODE  
    &ensp; : Crawling - 특일 정보 조회 (KASI)

  - DATE  
    &ensp; 2023-11-29 Created  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1)   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2)    
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3)   
    
 - DESC  
    &ensp; : 전처리 - 한국지역난방공사 열판매량/열공급량   
    &emsp; 1) 결측치가 없어서, 그대로 사용  
    &emsp;&ensp;&ensp; 
    &emsp;&ensp;&ensp; (Crawl Code 없음)   
    &emsp; 2) 

 - DATA  
    &emsp; <"Input">  
    1) None (Input Dataset)  
    &emsp;- Period :   
    &emsp;- Interval : 

    &emsp; <"Output">  
    1) Hourly (관측소/년도별 출력)  
    &nbsp;df_data_cal.to_csv(data_dir + 'KASI_DATE_D_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    2) Daily (관측소/년도별 출력)  
    &nbsp;df_data_cal_24.to_csv(data_dir + 'KASI_DATE_H_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['locdate', 'YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'HOUR', 'MINUTE']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    

 - Related Link  
    &ensp; : 

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [717]:
#region Basic_Import
## Basic
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import random
from pandas import DataFrame, Series

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score   

## Excel/CSV
import openpyxl, xlrd

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [718]:
## Import_DL
str_tar = "torch"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Torch Imported


In [719]:
## Import Model
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers import Dense, Flatten
# from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D

# multivariate data preparation
from numpy import array, hstack

In [720]:
## Import_Local (현재 프로그램에서 미사용)
from DEV_Common_Data_Analysis import print_desc_statistic
from DEV_Common_Data_Datetime import create_col_ymdhm, create_col_datetime, create_df_dt, conv_midnight_24to00, create_col_weekdays #, validate_date, list_invalidDate, calc_df_dt
from DEV_Common_Data_Preprocessing import resample_by_last, find_outlier_Usages, del_outlier_Usages
from DEV_Common_Data_Visualization import visualization_df
from DEV_KASI_Holiday import KASI_holiDay, KASI_restDay, KASI_anniDay
from DEV_KMA_Weather_ASOS import KMA_ASOS_DATA
from DEV_KDHC_Usage import KDHC_HEAT_Usage

### 01-01-02. Config (Directory, Params)

In [721]:
## Init_config
SEED = 42

np.random.seed(SEED)
# tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [722]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-01-05 18:59:03.908913
2024 / 1 / 5
18 : 59


## 01-02. Data Load (df_raw)

### 01-02-01. KDHC Heat Usage

In [723]:
# Define data root directory
str_dir_kdhcHeat = "./data_Energy_KDHC_National_Heat/"
# print(os.listdir(str_dir_kdhcHeat))

df_kdhc_heat = pd.read_csv(str_dir_kdhcHeat + 'KDHC_COMB_HEATxWEATHER_2019-2022.csv', index_col = 0)
df_kdhc_heat = df_kdhc_heat[['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'code_day_of_the_week', 'rest_YN', 'dist_from_holiday'
                             ,'HOUR'
                            #  , 'temp_outdoor', 'temp_dew_point', 'temp_ground'
                            #  , 'humidity'
                            #  , 'rainfall'
                            #  , 'snowfall', 'snowfall_3hr'
                            #  , 'wind_speed', 'wind_direction'
                            #  , 'pressure_vapor', 'pressure_area', 'pressure_sea'
                            #  , 'visual_range'
                             , 'PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG'
                             , 'KANGNAM'
                             , 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG', 'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG'
                             , 'KIMHAE', 'DAEGU', 'YANGSAN'
                             , 'GWANGJU_JEONNAM']]
## 어디서 중복 생긴건지 확인 필요
df_kdhc_heat = df_kdhc_heat.drop_duplicates()

df_kdhc_heat['METER_DATE'] = pd.to_datetime(df_kdhc_heat['METER_DATE'])
df_dt = df_kdhc_heat['METER_DATE']

## 추출된 "DATE"로부터 Week를 계산
# df_kdhc_heat['WEEK'] = df_kdhc_heat['METER_DATE'].dt.strftime('%G-%V') 

for i in range(0, len(df_kdhc_heat)):
    ## 중간에 Code화 되지 않은 rest_YN Data가 있다.
    if((str(df_kdhc_heat['rest_YN'].iloc[i]) == 'N') | (df_kdhc_heat['rest_YN'].iloc[i] == '0')):
        df_kdhc_heat['rest_YN'].iloc[i] = 0
    if((str(df_kdhc_heat['rest_YN'].iloc[i]) == 'Y') | (df_kdhc_heat['rest_YN'].iloc[i] == '1')):
        df_kdhc_heat['rest_YN'].iloc[i] = 1

    ## 중간에 이상하게 처리된 KASI Data가 있다.
    if((str(df_kdhc_heat['dist_from_holiday'].iloc[i]) == '제헌절')
        | (str(df_kdhc_heat['dist_from_holiday'].iloc[i]) == '한글날')):
        df_kdhc_heat['dist_from_holiday'].iloc[i] = 0

df_kdhc_heat['rest_YN'] = df_kdhc_heat['rest_YN'].astype('int64')
df_kdhc_heat['dist_from_holiday'] = df_kdhc_heat['rest_YN'].astype('int64')

print(df_kdhc_heat.columns)
# df_kdhc_heat

Index(['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'code_day_of_the_week', 'rest_YN',
       'dist_from_holiday', 'HOUR', 'PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG',
       'KANGNAM', 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON',
       'HWASUNG', 'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG', 'KIMHAE',
       'DAEGU', 'YANGSAN', 'GWANGJU_JEONNAM'],
      dtype='object')


In [724]:
# features_considered = ['PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG'
#                        , 'KANGNAM'
#                        , 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG', 'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG'
#                        , 'KIMHAE', 'DAEGU', 'YANGSAN'
#                        , 'GWANGJU_JEONNAM']
# features = df_kdhc_heat[features_considered]
# features.index = df_kdhc_heat['METER_DATE']

# features.plot(subplots=True)
# plt.show()

In [725]:
# fig, ax1 = plt.subplots(figsize=(30,5))

# ax1.plot(df_dt, df_kdhc_heat['PAJU'], color='red')

# ax2 = ax1.twinx()
# ax2.plot(df_dt, df_kdhc_heat['GOYANG'], color='green')

# plt.show()

In [726]:
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length):
        x = data.iloc[i:(i+seq_length)]
        y = data.iloc[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [738]:
## 휴일일 때
df_raw = df_kdhc_heat[df_kdhc_heat['rest_YN'] == 1]
col_tar = 'PAJU'
# col_tar = 'BUNDANG'
# col_tar = 'KANGNAM'

# df_kdhc_heat = df_kdhc_heat[[col_tar]]
df_raw = df_raw[['YEAR', 'MONTH', 'DAY', 'rest_YN','code_day_of_the_week', 'dist_from_holiday'
                , 'HOUR'
                , 'PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG'
                , 'KANGNAM', 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG', 'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG'
                , 'KIMHAE', 'DAEGU', 'YANGSAN'
                , 'GWANGJU_JEONNAM']]
df_raw

,YEAR,MONTH,DAY,rest_YN,code_day_of_the_week,dist_from_holiday,HOUR,PAJU,GOYANG,SAMSONG,...,SUWON,HWASUNG,DONGTAN,PYONGTAEK,CHEONGJU,SEJONG,KIMHAE,DAEGU,YANGSAN,GWANGJU_JEONNAM
94,2019,10,3,1,3,1,0,19,76.0000000000,18.0000000000,...,37.0000000000,27.0000000000,27,0.0000000000,29,34.0000000000,10,34.0000000000,12,4
96,2019,10,3,1,3,1,1,31,69.0000000000,17.0000000000,...,40.0000000000,19.0000000000,16,1.0000000000,21,23.0000000000,8,30.0000000000,11,4
98,2019,10,3,1,3,1,2,22,58.0000000000,18.0000000000,...,31.0000000000,17.0000000000,17,1.0000000000,18,16.0000000000,8,23.0000000000,8,3
100,2019,10,3,1,3,1,3,22,49.0000000000,14.0000000000,...,25.0000000000,18.0000000000,16,1.0000000000,17,18.0000000000,7,18.0000000000,8,3
102,2019,10,3,1,3,1,4,22,51.0000000000,10.0000000000,...,21.0000000000,19.0000000000,13,1.0000000000,15,17.0000000000,7,18.0000000000,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27178,2022,9,25,1,6,1,20,18,103.0000000000,44.0000000000,...,59.0000000000,43.0000000000,60,10.0000000000,32,56.0000000000,13,49.0000000000,23,14
27179,2022,9,25,1,6,1,21,19,95.0000000000,40.0000000000,...,61.0000000000,41.0000000000,58,9.0000000000,32,59.0000000000,14,39.0000000000,29,18
27180,2022,9,25,1,6,1,22,16,99.0000000000,34.0000000000,...,58.0000000000,35.0000000000,46,6.0000000000,37,57.0000000000,12,40.0000000000,20,17
27181,2022,9,25,1,6,1,23,13,85.0000000000,27.0000000000,...,48.0000000000,26.0000000000,37,6.0000000000,33,48.0000000000,12,33.0000000000,14,15


In [739]:
## Method A
# seq_length = 5
# X, y = create_sequences(df_kdhc_heat, seq_length)

# train_size = int(len(df_kdhc_heat) * 0.7)
# X_train, y_train = X[:train_size], y[:train_size]
# X_test, y_test = X[train_size:], y[train_size:]

## Method B
trainSet, testSet = train_test_split(df_raw, test_size=0.3, shuffle=False)

trainXX, trainYY = trainSet.drop([col_tar],axis=1), trainSet[[col_tar]]
testXX, testYY = testSet.drop([col_tar],axis=1), testSet[[col_tar]]

# trainXX

In [740]:
MIN_X, MAX_X = df_raw[col_tar].min(), df_raw[col_tar].max()
## MinMax 스케일링
def MinMaxScale(array, min, max):
    return (array - min) / (max - min)

X_train = MinMaxScale(trainXX, MIN_X, MAX_X)
y_train = MinMaxScale(trainYY, MIN_X, MAX_X)
X_test = MinMaxScale(testXX, MIN_X, MAX_X)
y_test = MinMaxScale(testYY, MIN_X, MAX_X)

## Tensor 형태로 변환
# def make_Tensor(array):
#     return torch.from_numpy(array).float()

# X_train = make_Tensor(X_train)
# y_train = make_Tensor(y_train)
# X_test = make_Tensor(X_test)
# y_test = make_Tensor(y_test)

# plt.plot(df_kdhc_heat.values)

### 1D CNN

In [741]:
# c = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=1, stride=1)

In [744]:
input = torch.Tensor([y_train[col_tar]])
output = c(input)
# output

w_list = []
for param in c.parameters():
    w_list.append(param)

w = w_list[0]
b = w_list[1]

w1 = w[0][0][0]

print(w1)
print(b)

tensor(-0.7397, grad_fn=<SelectBackward0>)
Parameter containing:
tensor([-0.4964], requires_grad=True)


In [735]:
## <PAJU>
## rest_YN = 0
## ▶ Weight
# tensor(-0.7397, grad_fn=<SelectBackward0>)
## ▶ Bias
# tensor([-0.4964], requires_grad=True)

## rest_YN = 1
## ▶ Weight
# tensor(0.9776, grad_fn=<SelectBackward0>)
# Parameter containing:
## ▶ Bias
# tensor([0.0176], requires_grad=True)

In [736]:
## <BUNDANG>
## rest_YN = 0
## ▶ Weight
# tensor(0.5419, grad_fn=<SelectBackward0>)
## ▶ Bias
# Parameter containing:
# tensor([-0.4190], requires_grad=True)

## rest_YN = 1
## ▶ Weight
# tensor(0.9776, grad_fn=<SelectBackward0>)
# Parameter containing:
## ▶ Bias
# tensor([0.0176], requires_grad=True)

In [737]:
print(stop)

NameError: name 'stop' is not defined

In [ ]:
df_kdhc_heat.describe().transpose()

In [ ]:
plt.hist2d(df_kdhc_heat['code_day_of_the_week'], df_kdhc_heat['PAJU'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('code_day_of_the_week')
plt.ylabel('PAJU')

In [ ]:
### 시간 데이터 변환

In [ ]:
# ## 시간 데이터 변환
# date_time = pd.to_datetime(df_kdhc_heat.pop('METER_DATE'), format='%d.%m.%Y %H:%M:%S')
# timestamp_s = date_time.map(pd.Timestamp.timestamp)

# day = 24*60*60
# year = (365.2425)*day

# df_kdhc_heat['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
# df_kdhc_heat['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
# df_kdhc_heat['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
# df_kdhc_heat['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
# df_kdhc_heat

In [ ]:
# plt.plot(np.array(df_kdhc_heat['Day sin'])[:25])
# plt.plot(np.array(df_kdhc_heat['Day cos'])[:25])
# plt.xlabel('Time [h]')
# plt.title('Time of day signal')

### 데이터 분할

In [ ]:
column_indices = {name: i for i, name in enumerate(df_kdhc_heat.columns)}

n = len(df_kdhc_heat)
train_df = df_kdhc_heat[0:int(n*0.7)]
# val_df = df_kdhc_heat[int(n*0.7):int(n*0.9)]
test_df = df_kdhc_heat[int(n*0.7):]

num_features = df_kdhc_heat.shape[1]

### 데이터 정규화

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
# val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
df_std = (df_kdhc_heat - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df_kdhc_heat.keys(), rotation=90)